In [2]:
# Sources
# https://github.com/rogerfitz/tutorials/tree/master/python_chess
# https://python-chess.readthedocs.io/en/latest/

     -------------------------------------- 149.1/149.1 kB 3.0 MB/s eta 0:00:00


In [4]:
!pip install chess
!pip install Stockfish

In [4]:
import chess.pgn
import pandas as pd
pd.options.display.max_columns=999
import datetime
import tqdm
import zipfile
import re

In [5]:
from stockfish import Stockfish
import chess
import chess.engine
import chess.svg
import chess.pgn
import re
import numpy as np
import pandas as pd
from statsmodels.distributions.empirical_distribution import ECDF
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline



In [26]:
NUM_GAMES=2386#from https://database.nikonoel.fr/ Not necessary but helpful to get tqdm to give estimate
rows=[]

with open(f'data/carlsen_games.pgn') as pgn:
    for game in tqdm.tqdm(range(NUM_GAMES)):
        row={}
        game = chess.pgn.read_game(pgn)
        row['headers']=game.headers.__dict__
        row['moves']=[x.uci() for x in game.mainline_moves()]
        rows.append(row)
games=pd.DataFrame(rows)
games
games.to_csv("data/carlsen.csv",index=False)


100%|█████████████████████████████████████████████████████████████████████████████| 2386/2386 [00:16<00:00, 145.65it/s]


In [8]:
def replace_numbers(string):
    pattern = r'\d'  # Matches any digit
    result = re.sub(pattern, lambda match: 'X' * int(match.group()), string)
    return result
def get_piece_position(fen, piece):
    # Split the FEN string into its components
    fen_parts = fen.split(' ')
    
    # Extract the board position part of the FEN
    board_fen = fen_parts[0]
    
    # Remove the slashes (/) to get a string of all the pieces
    board_fen = board_fen.replace('/', '')
    board_fen = replace_numbers(board_fen)    #Find the index of the knight (N) in the string
    indices = []
    for index, char in enumerate(board_fen):
        if char == piece:
            indices.append(index)
            break
            
    # Calculate the row and column based on the index
    try:
        row = 7 - (indices[0] // 8)
        column = indices[0] % 8
        return row, column
    except:
        return -1, -1
import pickle

def save_list(list_save, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(list_save, file)

def load_list(file_path):
    with open(file_path, 'rb') as file:
        chess_data = pickle.load(file)
    return chess_data

# Example usage
fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
knight_position = get_piece_position(fen, "N")
print("Knight Position: ", knight_position)

Knight Position:  (0, 1)


In [9]:
games=pd.read_csv("data/loaded_games.csv")
games['moves']=games['moves'].apply(lambda x: eval(x))
# games['headers']=games['headers'].apply(lambda x: eval(x))#to dict
games

,headers,moves
0,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","[d2d4, d7d6, g1f3, g8f6, c1f4, c8f5, e2e3, e7e..."
1,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","[d2d4, e7e5, e2e3, e5d4, e3d4, d7d5, f2f4, g8f..."
2,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","[e2e4, c7c5, g1f3, d7d6, d2d4, c5d4, f3d4, g8f..."
3,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","[d2d4, f7f5, g2g3, g8f6, f1g2, g7g6, g1f3, f8g..."
4,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","[d2d4, d7d5, c2c4, d5c4, e2e3, e7e5, f1c4, e5d..."
...,...,...
355530,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","[g1f3, d7d5, g2g3, c7c6, f1g2, g8f6, e1g1, c8g..."
355531,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","[g1f3, g8f6, c2c4, g7g6, b1c3, f8g7, e2e4, d7d..."
355532,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","[e2e4, c7c5, g1f3, b8c6, f1b5, e7e5, e1g1, d7d..."
355533,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","[e2e4, e7e5, g1f3, b8c6, f1c4, d7d6, b1c3, f8e..."


In [10]:
# df[a][b][c][d] = array of CPL of piece a at location (b, c) of color d
#0 = knight, a, 1, white
#df = np.zeros((2, 8, 8,2,1,), dtype=float)
df = [[[[[0]] * 2] * 8] * 8] * 2
#df
# df = np.arange(256)
# df = np.reshape(df, (2, 8, 8,2,1))
df[0][0][0][0]

[0]

In [11]:
engine = Stockfish("stockfish-windows-2022-x86-64-avx2.exe")
engine.set_depth(20)
engine.set_skill_level(20)
board=chess.Board()


In [12]:
# import numpy as np

# df = np.zeros((2, 8, 8,2,2), dtype=object)

In [13]:
# df[0][0][0][0].append(5)
# df[0][0][0][1]

In [14]:
# board=chess.Board()
# g
# g['moves'] = eval(g['moves'])

In [15]:
pieces = ['N', 'n', 'B', 'b']

In [16]:
df = []
for i in range(2):
    df.append([])
    for j in range(8):
        df[i].append([])
        for k in range(8):
            df[i][j].append([])
            for l in range(2):
                df[i][j][k].append([0])
df[1][7][7][1]

[0]

In [17]:
for i in tqdm.tqdm(range(1)):
    board=chess.Board()
    curr_game = games.iloc[i]
    print(curr_game['moves'][0])
    for move_number, san in enumerate(curr_game['moves']):
        if move_number == 10:
            break;
        move=board.push_san(san)
        engine.set_fen_position(board.fen())
        evaluation=engine.get_evaluation()
        for index, piece in enumerate(pieces):
            row, column = get_piece_position(board.fen(), piece)
            #print(board.fen())
            #print(i, row, column)
            if (row == -1):
                continue
            #print(evaluation)
            df[int(index / 2)][row][column][index % 2].append(evaluation['value'])


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

d2d4
rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1
0 0 1
{'type': 'cp', 'value': 32}
rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1
0 7 1
{'type': 'cp', 'value': 32}
rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1
0 0 2
{'type': 'cp', 'value': 32}
rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1
0 7 2
{'type': 'cp', 'value': 32}
rnbqkbnr/ppp1pppp/3p4/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 0 2
0 0 1
{'type': 'cp', 'value': 49}
rnbqkbnr/ppp1pppp/3p4/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 0 2
0 7 1
{'type': 'cp', 'value': 49}
rnbqkbnr/ppp1pppp/3p4/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 0 2
0 0 2
{'type': 'cp', 'value': 49}
rnbqkbnr/ppp1pppp/3p4/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 0 2
0 7 2
{'type': 'cp', 'value': 49}
rnbqkbnr/ppp1pppp/3p4/8/3P4/5N2/PPP1PPPP/RNBQKB1R b KQkq - 1 2
0 2 5
{'type': 'cp', 'value': 46}
rnbqkbnr/ppp1pppp/3p4/8/3P4/5N2/PPP1PPPP/RNBQKB1R b KQkq - 1 2
0 7 1
{'type': 'cp', 'value': 46}
rnbqkbnr/ppp1pppp/3p4/8/3P4/5N2/PPP1PPPP/RNBQKB1R

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.64s/it]

rn1qkb1r/ppp2pp1/3ppn1p/5b2/3P1B2/4PN1P/PPP2PP1/RN1QKB1R w KQkq - 0 6
0 2 5
{'type': 'cp', 'value': 44}
rn1qkb1r/ppp2pp1/3ppn1p/5b2/3P1B2/4PN1P/PPP2PP1/RN1QKB1R w KQkq - 0 6
0 7 1
{'type': 'cp', 'value': 44}
rn1qkb1r/ppp2pp1/3ppn1p/5b2/3P1B2/4PN1P/PPP2PP1/RN1QKB1R w KQkq - 0 6
0 3 5
{'type': 'cp', 'value': 44}
rn1qkb1r/ppp2pp1/3ppn1p/5b2/3P1B2/4PN1P/PPP2PP1/RN1QKB1R w KQkq - 0 6
0 7 5
{'type': 'cp', 'value': 44}


In [22]:
save_list(df, 'data/temp_list.pkl')

In [ ]:
# data[0]